In [254]:
using JLD, DSP, Statistics, SpecialFunctions
file = "/home/isabel/Dropbox/Proyects/Metodos_MEA/Scripts/Nuevos/spikes_140116s2cut2_evento_2.jld";
AllSpikes = load(file)["spikes"];
index, spikes = AllSpikes["2981"];
scales = 6;                        # number of scales for the wavelet decomposition 
inputs =  5;                       # number of inputs to the clustering 

In [256]:
wave_features( spikes, scales, inputs )

24×5 Matrix{Float64}:
  16.736     19.0482   19.9371     -9.11852     2.30755
   7.0076    30.2686  -13.9311     -6.39704     0.420403
  -9.20941   25.3843   -2.60032    18.8423    -20.6754
   3.52213   35.3211    3.55451   -31.0985      2.59777
 -13.0369    22.7587  -27.565     -26.9934     -1.78247
  13.8093    20.3775   -2.42487    22.2348     22.7041
   0.831946  27.1036   -9.57768   -16.9022    -37.3167
 -13.7926    27.4029   -9.17026     2.71926    11.764
  -7.18823   34.1862  -12.6826     -2.38605    26.6651
  33.3056    45.8073   -0.435502   -0.137795   33.066
 -10.1029    45.3507    4.53602    -5.92081   -18.0967
  52.2107    31.0363  -24.1763     40.1366     14.0186
 -13.0379    64.9802   44.305       4.24691   -24.416
 -17.2377    34.423   -17.3863      0.495062  -28.3109
 -11.7263    16.1983   22.8161      2.70503    19.2454
  -3.81387   34.5481   19.8065     -0.773893   24.1817
 -26.7273    43.263    -3.11423   -17.4038    -18.5654
  57.9233    28.4377   -5.61807   -18.594

In [217]:
function test_ks(x)
    # Calculates the CDF (expcdf)
    #[y_expcdf,x_expcdf]=cdfcalc(x);
    yCDF = [];
    xCDF = [];
    x = x[Bool.(1 .- isnan.(x))];
    n = length(x);
    x = sort(x);
    # Get cumulative sums
    yCDF = ((1:n)' )/n;
    # Remove duplicates; only need final one with total count
    notdup = ([diff(x); 1] .> 0);
    x_expcdf = x[notdup];
    y_expcdf = [0; yCDF[notdup]];
    # The theoretical CDF (theocdf) is assumed to be normal  
    # with unknown mean and sigma
    zScores  =  (x_expcdf .- mean(x))./std(x);
    #theocdf  =  normcdf(zScores , 0 , 1);
    mu = 0; 
    sigma = 1; 
    theocdf = 0.5 * erfc.(-(zScores.-mu)./(sqrt(2)*sigma));
    # Compute the Maximum distance: max|S(x) - theocdf(x)|.
    delta1    =  y_expcdf[1:end-1] .- theocdf;   # Vertical difference at jumps approaching from the LEFT.
    delta2    =  y_expcdf[2:end]   .- theocdf;   # Vertical difference at jumps approaching from the RIGHT.
    deltacdf  =  abs.(vcat(delta1,delta2));

    KSmax =  maximum(deltacdf);
    return KSmax 
end

test_ks (generic function with 1 method)

In [132]:
function wave_features( spikes, scales, inputs )
    nspk = size( spikes, 1 );
    ls = size( spikes, 2 );
    cc = zeros( nspk, ls );
    for i = 1:nspk                                
        c, l = fix_wavedec( spikes[ i, : ], scales );
        cc[ i, 1:ls ] = c[ 1:ls ];
    end
    sd = zeros( ls )
    for i = 1:ls                                  # KS test for coefficient selection   
        thr_dist = std( cc[ :, i ] ) * 3;
        thr_dist_min = mean( cc[ :, i ] ) - thr_dist;
        thr_dist_max = mean( cc[ :, i ] ) + thr_dist;
        aux = cc[ findall( ( cc[ :, i ] .> thr_dist_min ) .& ( cc[ :, i ] .< thr_dist_max ) ), i ];
        if length( aux ) > 10;
            ksstat = test_ks( aux );
            sd[ i ] = ksstat;
        else
            sd[ i ] = 0;
        end
    end
    coeff = zeros( inputs );
    max_sd = sort( sd );
    ind = sortperm( sd );
    coeff[ 1:inputs ] = ind[ ( ls - 1 ): -1: ls - inputs ];
    inspk = zeros( nspk, inputs );
    for i = 1:nspk
        for j = 1:inputs
            inspk[ i, j ] = cc[ i, Int( coeff[ j ] ) ];
        end
    end
    return inspk
end

function fix_wavedec( x, n )
    Lo_D = [ 0.7071, 0.7071 ];
    Hi_D = [ -0.7071, 0.7071 ];
    s = size( x ); 
    x = x'; 
    c = [ ]; 
    l = [ length( x ) ];
    shift = 0;
    for k = 1:n
        lf = length( Lo_D );
        lx = length( x );
        lenEXT = lf - 1; 
        lenKEPT = lx + lf - 1;
        I = getSymIndices( lx, lf );
        y = x[ I ];
        x = convdown( y, Lo_D, lenKEPT, shift );
        d = convdown( y, Hi_D, lenKEPT, shift );
        c = vcat( d, c );            # store detail
        l = vcat( length( d ), l );    # store length
    end
    c = vcat( x, c );
    l = vcat( length( x ), l );
    if s[ 1 ]>1 
        c = c'; 
        l = l'; 
    end
    return c, lmax.( deltacdf );
end

function getSymIndices( lx, lf )
I = vcat( collect( lf:-1:1 ) , collect( 1:lx ) , collect( lx:-1:lx-lf+1 ) )
    if lx < lf
        K = ( I < 1 );
        I[K] = 1-I( K ); ?max( )
        J = ( I > lx );
        while any( J )
            I[ J ] = 2*lx + 1 - I[ J ];
            K = ( I < 1 );
            I[K] = 1 - I[ K ];
            J = ( I > lx );
        end
    end
    return I
end


function convdown( q, f, lenKEPT, shift )
    p = conv2( q', f' ); 
    if size( q, 1 ) > 1 
        p = p'; 
    end
    sx = length( p );
    begInd = 1;
    first, last, ok = GetFirstLast( sx, begInd, lenKEPT );
    if ok 
        p = p[ Int( first[ 1 ] ):Int( last[ 1 ] ) ]; 
    end
    p = p[ 2 - rem( shift, 2 ):2:end ];
    return p
end

function GetFirstLast( sx, begInd, lenKEPT )

    oneDIM = isequal( begInd, 1 );
    s = lenKEPT;
    if ~oneDIM
        K  = find( s > sx );
        s[K] = sx[ K ];
        m = find( ( s < 0 )|( s != trunc( s ) ) );
        ok = isempty[ m ];
    else
        ok = ( s >= 0 )&( s < sx )&( s == trunc( s ) );
    end
    if ok == 0
        first = begInd; 
        last = s; 
    end
    nbarg = length( lenKEPT );
    if nbarg < 2 
        o = 'c'; 
    else 
        o = lowercase( lenKEPT[ 2 ] ); 
    end
    err = 0;
    if isletter( o[ 1 ] )
        case = o[ 1 ]
        if case == 'c'
            d = ( sx - s )/2;
            if nbarg < 3
                if length( o )>1 
                    side = o[ 2:end ]; 
                else 
                    side = 'l'; 
                end
            else
                side = lenKEPT[ 3 ];
            end
            if oneDIM
                first,last = GetFirst1D( side, sx, d );
            else
                if length( side ) < 2 
                    side[ 2 ] = 0; 
                end
                for k = 1:2
                    first[ k ], last[ k ] = GetFirst1D( side[ k ], sx[ k ], d[ k ] );
                end
            end

        elseif case == 'l'|'u' 
            first = begInd; 
            last = s;
        elseif case == 'r'|'d' 
                first = sx - s + 1; 
                last = sx;
        else      
            err = true;
        end
    else
        first = o; 
        last = first + s - 1;
        if !isequal( first, trunc( first ) ) | any( first < 1 ) | any( last > sx )
            err = true;
        end
    end
    if err == true
        println( "error" );
    end
    return first, last, ok
end

function GetFirst1D( side, s, d )
    
    if !isempty(indexin( side, [ 'u', 'l', '0', 0 ] ) )
        first = 1 + floor( d ); 
        last = s - ceil( d );
    elseif !isempty(indexin( side, [ 'd', 'r', '1', 1 ] ) ) 
        first = 1 + ceil( d );  
        last = s - floor( d );
    else
        first = 1 + floor( d ); 
        last = s - ceil( d ); 
    end
    return first, last
end

GetFirst1D (generic function with 1 method)